In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def check_bizfile_search_box_and_search(xlsx_path):
    if not os.path.exists(xlsx_path):
        print("❌ 找不到檔案：", xlsx_path)
        return

    # 讀取 Excel
    df = pd.read_excel(xlsx_path)
    if "公司名稱" not in df.columns:
        print("❌ Excel 檔案中缺少『公司名稱』欄")
        return

    # 若沒有「UEN 結果」欄，新增一欄
    if "UEN 結果" not in df.columns:
        df["UEN 結果"] = ""

    # Selenium 設定
    chrome_path = r"C:\Users\waylin\mydjango\e_invoice\upload\chromedriver.exe"
    options = Options()
    # options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service(executable_path=chrome_path)
    driver = webdriver.Chrome(service=service, options=options)

    try:
        for idx, row in df.iterrows():
            query = str(row["公司名稱"]).strip()
            if not query:
                continue

            print(f"\n🔍 處理第 {idx+1} 筆：{query}")
            driver.get("https://www.bizfile.gov.sg")

            # 等待搜尋框出現
            search_input = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.ID, "input-search-bar"))
            )

            driver.execute_script("arguments[0].scrollIntoView(true);", search_input)
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "input-search-bar"))
            )
            time.sleep(1)
            search_input.clear()

            # 模擬每個字元輸入
            for char in query:
                search_input.send_keys(char)
                time.sleep(0.3)

            search_input.send_keys(Keys.ENTER)
            time.sleep(1)
            search_input.send_keys(Keys.ENTER)

            # 擷取結果
            try:
                uen_elements = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".cmp-information-snippet-value-horizontal-container"))
                )

                if len(uen_elements) == 2:
                    uen = uen_elements[1].text.strip()
                    df.at[idx, "UEN 結果"] = uen
                    print(f"✅ 成功擷取 UEN：{uen}")

                elif len(uen_elements) < 2:
                    df.at[idx, "UEN 結果"] = "找不到UEN"
                    print(f"找不到 UEN")
                
                elif len(uen_elements) > 2:
                    df.at[idx, "UEN 結果"] = "找到多筆UEN"
                    print("⚠️ 結果超過兩間公司，請重新確認")
            except Exception as e:
                df.at[idx, "UEN 結果"] = "查詢失敗"
                print(f"❌ 查詢失敗：{e}")

            time.sleep(2)  # 每筆間隔，避免風控

    finally:
        driver.quit()
        # 儲存回同一份 Excel
        df.to_excel(xlsx_path, index=False)
        print(f"\n📄 所有結果已寫回 {xlsx_path}")

if __name__ == "__main__":
    xlsx_path = r"C:\Users\waylin\companies_search.xlsx"  # 改成你的實際路徑
    check_bizfile_search_box_and_search(xlsx_path)


🔍 處理第 1 筆：Advantech Co. Singapore Pte Ltd


C:\Windows\Temp\ipykernel_26028\692019245.py:75: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '199507032R' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "UEN 結果"] = uen


✅ 成功擷取 UEN：199507032R

🔍 處理第 2 筆：Shopee Singapore
✅ 成功擷取 UEN：Live Company

🔍 處理第 3 筆：DBS Bank Ltd
✅ 成功擷取 UEN：196800306E

🔍 處理第 4 筆：CAPITALAND GROUP PTE. LTD.
✅ 成功擷取 UEN：198900036N

🔍 處理第 5 筆：Keppel Corporation
✅ 成功擷取 UEN：196800351N

📄 所有結果已寫回 C:\Users\waylin\companies_search.xlsx
